In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('c://users//administrator//desktop')

In [2]:
s = pd.read_csv('products - Sheet1.csv')

In [182]:
s_check = pd.read_csv('products - Sheet1.csv')

In [151]:
s.head(2)

,ProductId,Title,Image URL,Division,Category,Sub category,Article type,Offer price,Sale price,Color,Gender
0,E8AC41D107EA5DBB5DF561A684899B38,Color Dose Pearls Bronze 59 Nail Polish,"https://media6.ppl-media.com/tr:h-750,w-750,c-...",Makeup,Nails,Nail Polish,Nail Polish,59,59,Color Dose,women
1,04AE770377AABCCBCCDD5FBC93CC83E1,Rimmel Space Dust Nail Polish - Luna Love #004,"https://media6.ppl-media.com/tr:h-750,w-750,c-...",Makeup,Nails,Nail Polish,Nail Polish,310,310,Rimmel,women


In [ ]:
# UNDERSTANDING AND CLEANING THE DATA

In [152]:
np.sum(s.isna())

ProductId          0
Title              0
Image URL          0
Division        2394
Category           0
Sub category       0
Article type       0
Offer price        0
Sale price         0
Color            108
Gender             0
dtype: int64

In [ ]:
## MORE SENSEFUL DATA IMPUTATION !!     NOT WORKING!

for i,row in s.iterrows():
    #print(row.index)
    if row['Division'] not in s.Division.value_counts().index:
        s.ix[i,'Division'] = s[s['Category'] == row['Category']]['Division'].unique()[0]
    else:
        pass
    if row['Color'] not in s.Color.value_counts().index:
        s.ix[i,'Color'] = s[s['Sub category'] == row['Sub category']]['Color'].unique()[0]
    else:
        pass

In [153]:
s.Division.value_counts()                      ## make is the most repeating category here

Makeup                   1357
Body                      820
Wellness                  682
Hair Care and Styling     552
Accessories               500
Fragrance                 292
Skin Care                 242
Men Grooming              235
Electronics               152
Name: Division, dtype: int64

In [154]:
## REPLACING ALL THE NA's with MODE of that column ( DIVISION)

s.Division.fillna(s.Division.value_counts().index[0],inplace = True)

In [155]:
## REPLACING ALL THE NA's with MODE of that column ( COLOR)

s.Color.fillna(s.Color.value_counts().index[0],inplace = True)

In [156]:
## TOO MANY COLORS IN COLORS COLUMN    ( FINDING COLORS THAT REPEATED FOR MORE THAN 50 TIMES)

top_colors = s.Color.value_counts()[s.Color.value_counts()>50].index                     ## MOST REPEATED IN THIS COLUMN IS CRUNCHY FASHION 
top_colors                                                                                 ## TOO MANY CATEGORIES

Index(['Crunchy Fashion', 'Faces', 'Healthvit', 'Good Vibes', 'herbal hills',
       'Inlife', 'Matrix Hair Color', 'Vaadi Herbals', 'Radico', 'he',
       'Gorgio Professional', 'Matrix', 'Veet', 'GlamGals', 'Essence',
       'Morpheme Remedies', 'Berina', 'Dettol', 'yardley', 'NourishVitals',
       'Colorbar', 'l'oreal paris', 'Karatcart', 'Blue Heaven', 'Whisper',
       'Ferosh', 'Bella', 'kronokare', 'Mond'Sub', 'Stay Quirky'],
      dtype='object')

In [157]:
## RETAINING TOP 50 COLORS

s['Color'] = [x if x in top_colors else 'other color' for x in s['Color']]

## EXTRACTING BRAND      from Title

In [158]:
def brand(x):
       return x.split(' ')[0]
    
s['brand'] = s.Title.apply(brand)

In [159]:
## TOO MANY BRAND IN COLORS COLUMN       ( FINDING BRANDS THAT GOT REPEATED MORE THAN 50 times)
 
top_brand = s.brand.value_counts()[s.brand.value_counts()>50].index                     ## MOST REPEATED IN THIS COLUMN IS CRUNCHY FASHION 
top_brand  

Index(['Crunchy', 'Matrix', 'Healthvit', 'Vaadi', 'Good', 'FACES', 'Faces',
       'Herbal', 'L'Oreal', 'Blue', 'Veet', 'Stay', 'INLIFE', 'Essence', 'LA',
       'Khadi', 'Bella', 'VLCC', 'Dettol', 'Radico', 'Gorgio', 'Colorbar',
       'Morpheme', 'GlamGals', 'Yardley', 'SSCPL', 'Herbalife', 'Lakme',
       'Berina', 'Elite', 'Whisper', 'Cheryl's', 'Karatcart', 'The',
       'Schwarzkopf', 'Ferosh', 'Gillette', 'Kronokare', 'NourishVitals'],
      dtype='object')

In [160]:
## TOP REPEATED SUB- CATEGORIES         ( TOO MANY SUBCATEGORIES)

top_subcat = s['Sub category'].value_counts()[s['Sub category'].value_counts()>=100].index
top_subcat

Index(['Face Cream', 'Deodorant Women', 'Eyeliner', 'Vitamins & Supplements',
       'Bangles & Bracelets', 'Fragrance For Men', 'Body Lotion', 'Face Blush',
       'Hair Clips & Accessories', 'Deodorant Men', 'Earrings', 'Rings',
       'Shampoo', 'Hand Sanitizer', 'Sponges & Wedges', 'Hand Wash',
       'Fragrance For Women', 'Massage Oils', 'Kajal and Kohl',
       'Matrix Hair Color Hair Color', 'Mascara', 'Hair Color',
       'Gift and Value Sets', 'Crunchy Fashion Jewellery', 'Handmade Soaps',
       'Regular Soaps', 'Foundation', 'Healthvit Family Nutrition', 'Powder',
       'Lipstick', 'Nail Polish', 'Concealer', 'Bronzers & Highlighters',
       'Radico Hair Color', 'Eyeshadow', 'Lip Gloss', 'Necklaces & Pendants',
       'Lip Liner', 'Family Nutrition'],
      dtype='object')

In [161]:
## RETAINING TOP 50 Sub - Categories

s['Sub category'] = [x if x in top_subcat else 'others' for x in s['Sub category']]

In [162]:
## RETAINING TOP 50 BRANDS

s['brand'] = [x if x in top_brand else 'other brand' for x in s['brand']]

## EXTRAACTING TYPE (SOLID,LIQUID AND COMBOS/OTHERS)  from Title

In [163]:
import re

def extracttype(l):
    x = re.search('ml' , l)                       ## ml
    if x:
        return 'liquid product'
    else:
        y = re.search(' g' , l)                     ## gms
        if y:
            return 'solid product'
        else:
            z = re.search('Combo' , l)               ## combo
            if z:
                return 'combo product'
            else:
                return 'other'

In [164]:
s['type'] = s.Title.apply(extracttype)

In [165]:
s.type.value_counts()

other             2702
liquid product    2420
solid product     1994
combo product      110
Name: type, dtype: int64

In [171]:
### DROPPING UNWANTED COLUMNS AND SETTING INDEX 

s.set_index(['ProductId'],inplace=True)
s.drop(['Title','Image URL'],axis= 1,inplace=True)

,Division,Category,Sub category,Article type,Offer price,Sale price,Color,Gender,brand,type
ProductId,,,,,,,,,,
E8AC41D107EA5DBB5DF561A684899B38,Makeup,Nails,Nail Polish,Nail Polish,59,59,other color,women,other brand,other
04AE770377AABCCBCCDD5FBC93CC83E1,Makeup,Nails,Nail Polish,Nail Polish,310,310,other color,women,other brand,other
21B0CF144B3CFFDDCCE4937C7731A7CE,Makeup,Nails,Nail Polish,Nail Polish,59,59,other color,women,other brand,other
129CE0C7596D2D346E5CDC54E55A3A67,Makeup,Nails,Nail Polish,Nail Polish,59,59,other color,women,other brand,other
A5E51E0F6C9929ECD16D875EC9A4AFA5,Makeup,Nails,Nail Polish,Nail Polish,237,249,other color,women,other brand,other


## USING LABEL ENCODER

In [172]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

s_le = s.apply(le.fit_transform)

In [173]:
s_le.head()

,Division,Category,Sub category,Article type,Offer price,Sale price,Color,Gender,brand,type
ProductId,,,,,,,,,,
E8AC41D107EA5DBB5DF561A684899B38,5,27,30,75,36,25,29,1,39,2
04AE770377AABCCBCCDD5FBC93CC83E1,5,27,30,75,273,185,29,1,39,2
21B0CF144B3CFFDDCCE4937C7731A7CE,5,27,30,75,36,25,29,1,39,2
129CE0C7596D2D346E5CDC54E55A3A67,5,27,30,75,36,25,29,1,39,2
A5E51E0F6C9929ECD16D875EC9A4AFA5,5,27,30,75,207,155,29,1,39,2


#  MODEL BUILDING   (KNN)

In [175]:
from scipy.spatial.distance import euclidean as ed

# INPUT

In [203]:
## TAKING ONE OF THE Product-Id as input

## EX:  taking the first product

pid = 'E8AC41D107EA5DBB5DF561A684899B38'

ref = s_le.loc['E8AC41D107EA5DBB5DF561A684899B38',:]            ## REFERENCE ROW TO GET RECOMENDATIONS

In [204]:

## CALCULATING THE DISTANCE OF EACH OTHER PRODUCT FROM THE PRESENT PRODUCT

d ={}
for i,row in s_le.iterrows():
    d[i] = round(ed(np.array(row),np.array(ref)),2)

In [205]:
## MAKING A DISTANCE DATAFRAME

dist = pd.DataFrame([np.array(list(d.keys())),np.array(list(d.values()))],index=['index','distance'])
dist = dist.T

In [206]:
##  5 NEAREST PRODUCTS 
dist.sort_values('distance').head(10)

,index,distance
0,E8AC41D107EA5DBB5DF561A684899B38,0
2,21B0CF144B3CFFDDCCE4937C7731A7CE,0
3,129CE0C7596D2D346E5CDC54E55A3A67,0
86,4BECE00CF75D77C061BD06EF469B799F,8.25
37,B7F1B941435297C9097DD67247A4731A,8.25
6936,6C44B35677DA4F58F1CA6A0FDD755AAA,15.13
52,C0DCC879E06A224650B1C4176A3F9F80,15.81
2400,0C671A66F2236BA11BD96F2B5F9C9AB5,15.84
5442,76AB36DB96146022986774BA94207E7F,16.58
976,4BA425C158226962FF4495BD4C230FC8,17.38


In [207]:
nearest_products = dist.sort_values('distance')['index'].head(10)
nearest_products

0       E8AC41D107EA5DBB5DF561A684899B38
2       21B0CF144B3CFFDDCCE4937C7731A7CE
3       129CE0C7596D2D346E5CDC54E55A3A67
86      4BECE00CF75D77C061BD06EF469B799F
37      B7F1B941435297C9097DD67247A4731A
6936    6C44B35677DA4F58F1CA6A0FDD755AAA
52      C0DCC879E06A224650B1C4176A3F9F80
2400    0C671A66F2236BA11BD96F2B5F9C9AB5
5442    76AB36DB96146022986774BA94207E7F
976     4BA425C158226962FF4495BD4C230FC8
Name: index, dtype: object

# 10 NEAREST PRODUCTS TO 
PRODUCT_ID :'E8AC41D107EA5DBB5DF561A684899B38' and Product: 'Color Dose Pearls Bronze 59 Nail Polish' are :

In [208]:
for pid in nearest_products:
    print(s_check[s_check['ProductId'] == pid]['Title'])

0    Color Dose Pearls Bronze 59 Nail Polish
Name: Title, dtype: object
2    Color Dose Pearls Copper Rust 58 Nail Polish
Name: Title, dtype: object
3    Color Dose Matt Nude 25 Nail Polish
Name: Title, dtype: object
86    Stay Quirky Nail Polish, It'S Pinke 466 (6 ml)
Name: Title, dtype: object
37    Stay Quirky Nail Polish, Pink-Iness 403 (6 ml)
Name: Title, dtype: object
6936    Kaiv Safety Pin Color SFP3303
Name: Title, dtype: object
52    Stay Quirky Nail Polish, Fresh Water Lake 518 ...
Name: Title, dtype: object
2400    Divo Set of 2 Foundation Round Sponges
Name: Title, dtype: object
5442    GUBB USA Cotton Balls 50S
Name: Title, dtype: object
976    Dove Cream Beauty Bathing Bar (75 g)
Name: Title, dtype: object
